In [2]:
%run "__imports__.ipynb"


def get_zenodo_entries(query):

    ZENODO_ACCESS_TOKEN = 'k3Sb7fEuIpBB41nXG7RQyXJhp77mWZtp4mmUahMQasDXOPHyLdjpgGiPLbGF'

    # Get articles from zenodo from a search query and compare them with the ones already saved
    r = requests.get('https://zenodo.org/api/records/',
                     params={'access_token': ZENODO_ACCESS_TOKEN, 'q' : "\"" + query + "\""})

    database = r.json()
    today =  datetime.datetime.combine(datetime.date.today(), datetime.time(0, 0))

    total_articles = []
    
    while (r.status_code == 200) :
        for article in database['hits']['hits']:
            # check if the article has an identifier
            if not 'doi' in article:
                continue
            # check if the article is open and can be downloaded
            if (article['metadata']['access_right'] == 'embargoed'):
                embargo_date =  datetime.datetime.strptime(article['metadata']['embargo_date'], '%Y-%m-%d')
                if (embargo_date > today):
                    continue
            elif (article['metadata']['access_right'] != 'open'):
                continue

            # if there is metadata missing, go to the next entry
            if 'files' not in article:
                continue
            #keep only pdf files, since they can be transformed to text
            if article['files'][0]['type'] != 'pdf':
                continue
            total_articles.append(article)
            
            
        # go to the next page of results
        if 'next' in database['links']:
            r = requests.get(database['links']['next'], params={'access_token': ZENODO_ACCESS_TOKEN})
            database = r.json()

        else:
            break

    return total_articles
    

    
# Download the PDF file, pre-process the text and save it in the text file
def save_pdf_and_get_text (link_to_download, filename=TEMP_FILE_PDF):

    keywords = []
    response = requests.get(link_to_download)

    # keep abbreviations, they will be replaced after cleanup
    # by the corresponding words sequence
    abbr_lower = [abbr.lower() for abbr in abbreviations]
    with open(filename, 'wb') as f:
        f.write(response.content)

    text = cmdExtractPdfText()
    text = prepareDescription(text, keepwords, abbr_lower)
    text = replace_abbrevations(text, abbreviations)
    return text.split(' ')


# Call the pdftotext Linux utility to transform the content of the pdf file into text
# This is the fastest way of getting ata
def cmdExtractPdfText(filePath=TEMP_FILE_TXT):
    subprocess.call(("pdftotext " + TEMP_FILE_PDF + " " + filePath).split())
    text = ""
    with open(filePath, encoding='utf-8') as f:
        text = f.read()
    
    return text


# This is the alternative way, there is a PyPDF2 module in python that read files,
# but it doesn't get them well everytime. It also requires qpdf utulity to repaid the files,
# since at the first try, it returns an error, considering the file corrupt
def extractPdfText(filePath=''):

    # Open the pdf file in read binary mode.
    fileObject = open(filePath, 'rb')

    try:
        # Create a pdf reader .
        pdfFileReader = PyPDF2.PdfFileReader(fileObject, strict=False)
    
    except:
        subprocess.call(("qpdf " + TEMP_FILE_PDF + " " + TEMP_FILE +".repaired.pdf").split())
        subprocess.call("cp " + TEMP_FILE +".repaired.pdf" + TEMP_FILE +".repaired.pdf".split())
        pdfFileReader = PyPDF2.PdfFileReader(fileObject, strict=False)

    text = ''
        
    # Get total pdf page number.
    try:
        totalPageNumber = pdfFileReader.numPages
    except:
        return text

    # Print pdf total page number.
    print('This pdf file contains totally ' + str(totalPageNumber) + ' pages.')

    currentPageNumber = 0

    # Loop in all the pdf pages.
    while(currentPageNumber < totalPageNumber):

        # Get the specified pdf page object.
        try:
            pdfPage = pdfFileReader.getPage(currentPageNumber)
            # Get pdf page text.
            # print(str(pdfPage.extractText()))
            page_text = ''
            try:
                page_text = str(pdfPage.extractText())
            except:
                print("Cannot extract text from page %d" %currentPageNumber)
            text = text + ' ' + page_text
        except:
            print("No content page %d" %currentPageNumber)

        # Process next page.
        currentPageNumber += 1
    
    return text


In [3]:
"""
results = get_zenodo_entries("solar irradiance")

documents = []

documents.extend(get_zenodo_entries("photovoltaic"))
documents.extend(get_zenodo_entries("Renewable Energy"))
documents.extend(get_zenodo_entries("solar pond"))
documents.extend(get_zenodo_entries("solar observations"))

search_doi = [article['doi'] for article in results]
database_doi = [article['doi'] for article in documents]
    
database_doi = [doi for doi in database_doi if doi not in search_doi]

# we already have the fies downloaded
# so look for them in the /tmp folder
# otherwise, download it
for doc in documents:
    if doc['doi'] in database_doi:
        mod_doi = doc['doi'].replace('/', '-')

        #look in tmp folder if there is a file containing the doi
        # if there is, just read the file and move to the next entry
        if os.path.isfile('/tmp/' + mod_doi + ".txt"):
            continue

        doc_list = save_pdf_and_get_text(doc['files'][0]['links']['self'])
        # overwrite the doi temporary file
        with open('/tmp/' + mod_doi + ".txt", 'w') as f:
            f.write(' '.join(doc_list))
            f.close()

#get each file in the folder and build a corpus based on those articles
corpus = []
for doi in database_doi:
    mod_doi = doi.replace('/', '-')
    with open('/tmp/' + mod_doi + ".txt", 'r') as f:
        corpus.append(f.read().split())
    
print(corpus)

"""



'\nresults = get_zenodo_entries("solar irradiance")\n\ndocuments = []\n\ndocuments.extend(get_zenodo_entries("photovoltaic"))\ndocuments.extend(get_zenodo_entries("Renewable Energy"))\ndocuments.extend(get_zenodo_entries("solar pond"))\ndocuments.extend(get_zenodo_entries("solar observations"))\n\nsearch_doi = [article[\'doi\'] for article in results]\ndatabase_doi = [article[\'doi\'] for article in documents]\n    \ndatabase_doi = [doi for doi in database_doi if doi not in search_doi]\n\n# we already have the fies downloaded\n# so look for them in the /tmp folder\n# otherwise, download it\nfor doc in documents:\n    if doc[\'doi\'] in database_doi:\n        mod_doi = doc[\'doi\'].replace(\'/\', \'-\')\n\n        #look in tmp folder if there is a file containing the doi\n        # if there is, just read the file and move to the next entry\n        if os.path.isfile(\'/tmp/\' + mod_doi + ".txt"):\n            continue\n\n        doc_list = save_pdf_and_get_text(doc[\'files\'][0][\'links